# Data Utilities

This is a library of utilities which we use for exploratory data analysis on the "Mixed Signals" MNIST data.



In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt
import h5py
import umap
from sklearn import manifold

In [2]:
def load_data(fname):
    """Load dataset and labels from HDF5 file with properties (X, labels)."""
    with h5py.File(fname, 'r') as f:
        X = f["X"].value
        try:
            labels = f["labels"].value
        except KeyError:
            labels = None
    return X, labels

In [17]:
def normalize_image_data(X):
    """
    Normalize image data through the following process:
    
    1. Convert pixel intensities (0-255) to floats (0.0-1.0).
    2. Mean center the data.
    3. Compute the covariance.
    
    Returns:
    
    * X0 - the mean centered data
    * Xbar - mean values
    * CX - the mean centered covariance matrix
    """
    X = X / 255.0
    N = X.shape[0]
    Xbar = np.mean(X, 0)
    X0 = X - Xbar
    CX = (X0.T @ X0) / (N - 1)
    return X0, Xbar, CX

In [4]:
def X_to_img(X, Xbar):
    """
    Transform from mean-centered data space (X) back to pixel space.
    This should always be done prior to calling `show_img`.
    """
    return np.asarray(np.clip((X + Xbar), 0.0, 1.0) * 255.0, dtype=np.uint8)

In [5]:
def graph_variance_explained(S, n=100):
    """
    Produce a graph of the covariance explained by principal components from `S`,
    the vector representation of the diagonal matrix of singular values
    resulting from SVD on the covariance matrix.
    """
    S_cum = np.add.accumulate(S)
    S_tot = S_cum[-1]
    expl = S_cum / S_tot
    X = np.arange(1,n+1)
    plt.figure(figsize=(10,6))
    plt.plot(X, expl[:n], label="Total Variance Explained")
    plt.plot(X, S[:n] / S[0], label="Singular Value (% of first)")
    plt.legend()

In [6]:
def show_img(x):
    """Show a single image by reshaping to 28x28 and plotting the values."""
    fig = plt.imshow(x.reshape(28,28), vmin=0, vmax=255, cmap='viridis')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    return fig

In [7]:
def show_data(X, cols=5, width=10):
    """Show a sample of data values represented by the rows of `X` as a grid of images."""
    n = min(len(X), 200)
    rows = 1 + (n//cols)
    height = (width / float(cols)) * rows
    plt.figure(figsize=(width,height))
    for i in range(n):
        plt.subplot(rows,cols,i+1)
        fig = show_img(X[i].reshape(28,28))
    plt.tight_layout()
    plt.subplots_adjust(wspace=0, hspace=0)

In [8]:
def show_pcs(U, n=10, start=1):
    """
    Display the first `n` principal components as images.
    These are taken as columns of the `U` matrix from the SVD.
    """
    rows = 1 + (n//5)
    plt.figure(figsize=(15,rows*2))
    for i in range(start-1, start+n-1):
        plt.subplot(rows,5,i-start+2)
        fig = plt.imshow(U[:,i].reshape(28,28), cmap='PRGn')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.title("PC {}".format(i+1))
    plt.tight_layout()

In [9]:
def show_splom(X, labels=None, width=10):
    """
    Create a categorically colored scatterplot matrix with stacked histograms on the diagonal.
    
    Based on [Stackoverflow solution](https://stackoverflow.com/questions/7941207/is-there-a-function-to-make-scatterplot-matrices-in-matplotlib).
    """
    N, D = X.shape
    nbins = 50
    fig, axes = plt.subplots(nrows=D, ncols=D, figsize=(width,width))
    fig.subplots_adjust(hspace=0.05, wspace=0.05)

    for ax in axes.flat:        
        # Hide all ticks and labels
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)

        # Set up ticks only on one side for the "edge" subplots...
        if ax.is_first_col():
            ax.yaxis.set_ticks_position('left')
        if ax.is_last_col():
            ax.yaxis.set_ticks_position('right')
        if ax.is_first_row():
            ax.xaxis.set_ticks_position('top')
        if ax.is_last_row():
            ax.xaxis.set_ticks_position('bottom')

    # Plot the data.
    for i, j in zip(*np.triu_indices_from(axes, k=1)):
        for x, y in [(i,j), (j,i)]:
            axes[x,y].scatter(X[:,y], X[:,x], c=labels, cmap='tab10', s=1)

    uniq_labels = np.sort(np.unique(labels))
    stackedX = [X[labels==i] for i in uniq_labels]
    # Use 1-D histograms on diagonal plots
    for i in range(D):
        component = [a[:,i] for a in stackedX]
        axes[i,i].hist(component, nbins, histtype='bar', stacked=True)

#     # Turn on the proper x or y axes ticks.
#     for i, j in zip(range(numvars), itertools.cycle((-1, 0))):
#         axes[j,i].xaxis.set_visible(True)
#         axes[i,j].yaxis.set_visible(True)

In [10]:
def show_embeddings(Xlist, titles=None, labels=None):
    """
    Display multiple embeddings as a horizontal strip of 2-D scatterplots.
    
    Inputs:
    * Xlist - list of embedding matrices, each with shape (N,2)
    * titles - corresponding list of plot titles (optional)
    * labels - array labels (for coloring only)    
    """
    num = len(Xlist)
    plt.figure(figsize=(5*num,5))
    for i, X in enumerate(Xlist):
        plt.subplot(1,num,i+1)
        plt.scatter(X[:,0], X[:,1], c=labels, cmap='tab10', s=1)
        plt.axis('equal')
        plt.xticks([])
        plt.yticks([])
        if titles:
            plt.title(titles[i])
    plt.tight_layout()

In [11]:
def isomap_embedding(X, n_neighbors=15):
    """Compute the 2-D Isomap embedding of the given data."""
    return manifold.Isomap(n_neighbors=n_neighbors, n_components=2).fit_transform(X)

In [12]:
def lle_embedding(X, n_neighbors=15):
    """Compute the 2-D LLE embedding of the given data."""
    return manifold.LocallyLinearEmbedding(n_neighbors=n_neighbors, n_components=2).fit_transform(X)

In [13]:
def mlle_embedding(X, n_neighbors=15):
    """Compute the 2-D LLE embedding of the given data."""
    return manifold.LocallyLinearEmbedding(n_neighbors=n_neighbors, n_components=2, method='modified').fit_transform(X)

In [14]:
def tsne_embedding(X, perplexity=15.0):
    """Compute the tSNE embedding of the given data."""
    return manifold.TSNE(perplexity=perplexity, n_components=2).fit_transform(X)

In [15]:
def umap_embedding(X, n_neighbors=15):
    """Compute the 2-D UMAP embedding of the given data."""
    return umap.UMAP(n_neighbors=n_neighbors, n_components=2).fit_transform(X)

In [16]:
def get_cpcs(Acov, Bcov, alpha=1.0):
    sigma = Acov - Bcov * alpha
    w, v = np.linalg.eig(sigma)
    idxs = np.argsort(-w)
    vtop = v[:,idxs]
    return vtop